In [4]:
# !pip install dask-ml

In [5]:
from dask.distributed import Client
client = Client(n_workers=2, threads_per_worker=1)
client

/usr/local/lib/python3.10/dist-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46075 instead
  warnings.warn(
INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:36937
INFO:distributed.scheduler:  dashboard at:  http://127.0.0.1:46075/status
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:45479'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:46675'
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:39697', name: 0, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:39697
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:45180
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:37701', name: 1, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:46075/status,
Dashboard: http://127.0.0.1:46075/status,Workers: 2
Total threads: 2,Total memory: 12.67 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:36937,Workers: 2
Dashboard: http://127.0.0.1:46075/status,Total threads: 2
Started: Just now,Total memory: 12.67 GiB
Comm: tcp://127.0.0.1:39697,Total threads: 1
Dashboard: http://127.0.0.1:42697/status,Memory: 6.34 GiB
Nanny: tcp://127.0.0.1:45479,


In [6]:
import dask
import dask.array as da
from dask_ml.datasets import make_classification


n, d = 100000, 100

X, y = make_classification(n_samples=n, n_features=d,
                           chunks=n // 100, flip_y=0.2)
X


/usr/local/lib/python3.10/dist-packages/dask/base.py:1437: UserWarning: Running on a single-machine scheduler when a distributed client is active might lead to unexpected results.
  warnings.warn(


dask.array<normal, shape=(100000, 100), dtype=float64, chunksize=(1000, 100), chunktype=numpy.ndarray>

In [7]:
from dask_ml.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train

dask.array<concatenate, shape=(90000, 100), dtype=float64, chunksize=(900, 100), chunktype=numpy.ndarray>

In [8]:
X_train, X_test, y_train, y_test = dask.persist(X_train, X_test, y_train, y_test)

In [9]:
classes = da.unique(y_train).compute()
classes

array([0, 1])

In [10]:
from sklearn.linear_model import SGDClassifier

est = SGDClassifier(loss='log', penalty='l2', tol=1e-3)

In [11]:
# Wrap with Dask ML
from dask_ml.wrappers import Incremental

inc = Incremental(est, scoring='accuracy')

In [12]:
# Training
inc.fit(X_train, y_train, classes=classes)

Incremental(estimator=SGDClassifier(loss='log'), scoring='accuracy')

In [13]:
inc.score(X_test, y_test)

0.6057